In [14]:
# Import libraries
import base58
import bech32
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

In [15]:
# Useful functions

def dSHA256(data):
    '''Two rounds of SHA256 (aka Hash256)'''
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    '''Converts a private key (bytes) to a compressed pubkey (bytes)'''
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

def pk_to_p2wpkh(compressed, network):
    '''generates a p2wpkh bech32 address corresponding to a compressed pubkey'''
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    spk = binascii.unhexlify(redeemScript.hex())
    version = spk[0] - 0x50 if spk[0] else 0
    program = spk[2:]
    if network == "testnet":
        prefix = 'tb'
    if network == "regtest":
        prefix = 'bcrt'
    elif network == "simnet":
        prefix = 'sb'
    elif network == "mainnet":
        prefix = 'bc'
    else:
        return "Enter the network: tesnet/simnet/mainnet"
    return bech32.encode(prefix, version, program)

# Bitcoin Core Regtest P2WPKH tx Demo

This notebook is starts up Bitcoin Core in regtest mode and walks through an example of creating and broadcasting a P2WPKH transaction with one input and two outputs. The transaction is generated using the ```P2WPKH_to_P2WPKH.py``` script.

This demo assumes you have Bitcoin Core (v0.18 or higher) and that the application data is stored in 
```$HOME/Library/Application Support/Bitcoin```.

#### Create private keys and bech32 addresses for the sender, receiver, and change outputs.

In [16]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2wpkh_addr = pk_to_p2wpkh(sender_pubkey, network = "regtest")
print("sender's p2wpkh address: " + sender_p2wpkh_addr)

change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)
change_p2wpkh_addr = pk_to_p2wpkh(change_pubkey, network = "regtest")
print("sender's change p2wpkh address: " + change_p2wpkh_addr)

receiver_privkey = bytes.fromhex("3333333333333333333333333333333333333333333333333333333333333333")
receiver_pubkey = privkey_to_pubkey(receiver_privkey)
receiver_p2wpkh_addr = pk_to_p2wpkh(receiver_pubkey, network = "regtest")
print("receiver's p2wpkh address: " + receiver_p2wpkh_addr)

sender's p2wpkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw
sender's change p2wpkh address: bcrt1q2vfxp232rx0z9rzn0hay9jptagk8c86ddphpjv
receiver's p2wpkh address: bcrt1q80pg6mvjmyrnld0r4h6gz7274azxhnhdf7k5gu


#### Start up regtest mode, delete any regtest network history so we are starting from scratch. Mine 101 blocks so that we can spend some bitcoin.

In [21]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -fallbackfee=0.0002")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
address = subprocess.getoutput("bitcoin-cli -regtest getnewaddress")
result = subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 101 {addr}".format(addr=address))
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

# Check that we were able to mine 101 blocks
print("blockcount: " + str(blockcount))

blockcount: 101


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [22]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + sender_p2wpkh_addr + " 2.001")
print(txid_1)

7f6956d404090f41fcb61b22b5b658517933cc31a3e8a932fa0372b43fe62646


#### Find which output index the btc was sent to

In [24]:
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
# raw_tx
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
json = json.loads(decoded)
print(decoded)

if json["vout"][0]["scriptPubKey"]["addresses"][0] == sender_p2wpkh_addr:
    index = 0
else:
    index = 1
print("index: " + str(index))

{
  "txid": "7f6956d404090f41fcb61b22b5b658517933cc31a3e8a932fa0372b43fe62646",
  "hash": "18c9089da6cf1f3dcb5aa0ea7df6a090853f7d9575b6199fac8fd1fa3bde1063",
  "version": 2,
  "size": 222,
  "vsize": 141,
  "weight": 561,
  "locktime": 101,
  "vin": [
    {
      "txid": "dd4be5a6c1346561cefb6d9f499422acf0973f0d8726af4d383a132fafd90b83",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3044022038467d12104ecc96b8f645dbd746cf50673f5247ecb1219014e1410c96e153270220159c718d639129c63686dfd633fe18940f83de610bdf381e3f2227e2304c635601",
        "02130f54a11e76e99e4cb1d8b7674696da608368d3872c9a8867f16676f558e599"
      ],
      "sequence": 4294967294
    }
  ],
  "vout": [
    {
      "value": 47.99897180,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 4bb5e1832d80176ea2aba7a56662a7e3544b27ec",
        "hex": "00144bb5e1832d80176ea2aba7a56662a7e3544b27ec",
        "reqSigs": 1,
        "type": "witness_v0_keyhash",
     

#### Mine a block so that the funding tx gets confirmed

In [25]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

#### Generate the P2WPKH to P2WPKH tx using our script.

In [26]:
new_tx = subprocess.getoutput("python P2WPKH_to_P2WPKH.py" 
                        + " --txid_str " + txid_1
                        + " --index " + str(index)
                        + " --input_amount_btc " + "2.001"
                        + " --privkey " + sender_privkey.hex()
                        + " --output1_pubkey " + receiver_pubkey.hex()
                        + " --output1_value_btc " + "1.5"
                        + " --output2_pubkey " + change_pubkey.hex()
                        + " --output2_value_btc " + "0.5")
print(new_tx)

020000000001014626e63fb47203fa32a9e8a331cc33795158b6b5221bb6fc410f0904d456697f0100000000ffffffff0280d1f008000000001600143bc28d6d92d9073fb5e3adf481795eaf446bceed80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d02473044022043f99ea2bbc1b71c2c6ef1c05aca61449be9ecbc4524150fb73327ae090d3d08022002e333a57636669a0de0a3dcaa5b407500adf882208cacf71cce5d5f0d3d90030121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


#### Broadcase the transaction (on regtest mode). If we get back a txid (32 byte hash), then it means the tx was successfully broadcast!

In [27]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + new_tx)
print(new_tx_txid)

314be179c321ea4fd43d2477b625c425ff68e459e7301d0ad19d50249094a4f1


#### We can search for the raw transaction in the blockchain using the command ```getrawtransaction``` and the txid.

In [29]:
raw_broadcasted_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + new_tx_txid)
print(raw_broadcasted_tx)

020000000001014626e63fb47203fa32a9e8a331cc33795158b6b5221bb6fc410f0904d456697f0100000000ffffffff0280d1f008000000001600143bc28d6d92d9073fb5e3adf481795eaf446bceed80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d02473044022043f99ea2bbc1b71c2c6ef1c05aca61449be9ecbc4524150fb73327ae090d3d08022002e333a57636669a0de0a3dcaa5b407500adf882208cacf71cce5d5f0d3d90030121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


#### We can decode the serialized transaction using ```decoderawtransction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [30]:
print("receiver's p2wpkh address: " + receiver_p2wpkh_addr)
print("sender's change p2wpkh address: " + change_p2wpkh_addr)

receiver's p2wpkh address: bcrt1q80pg6mvjmyrnld0r4h6gz7274azxhnhdf7k5gu
sender's change p2wpkh address: bcrt1q2vfxp232rx0z9rzn0hay9jptagk8c86ddphpjv


In [31]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_broadcasted_tx)
print(decoded)

{
  "txid": "314be179c321ea4fd43d2477b625c425ff68e459e7301d0ad19d50249094a4f1",
  "hash": "656616282ea4b3317b7b08f5980e19611707e17dc0c6fc0621df9eeae9500b43",
  "version": 2,
  "size": 222,
  "vsize": 141,
  "weight": 561,
  "locktime": 0,
  "vin": [
    {
      "txid": "7f6956d404090f41fcb61b22b5b658517933cc31a3e8a932fa0372b43fe62646",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3044022043f99ea2bbc1b71c2c6ef1c05aca61449be9ecbc4524150fb73327ae090d3d08022002e333a57636669a0de0a3dcaa5b407500adf882208cacf71cce5d5f0d3d900301",
        "034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 3bc28d6d92d9073fb5e3adf481795eaf446bceed",
        "hex": "00143bc28d6d92d9073fb5e3adf481795eaf446bceed",
        "reqSigs": 1,
        "type": "witness_v0_keyhash",
        

#### Stop bitcoin core running in the background.

In [32]:
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'